In [1]:
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from sparknlp.annotator import StopWordsCleaner
from pyspark.sql import types as T
from pyspark.sql import functions as F
from sparknlp.annotator import NGramGenerator
from sparknlp.base import Finisher
from pyspark.ml.tuning import ParamGridBuilder

spark = sparknlp.start()

data = spark.read.csv("../data_topicmodel.csv", header= True).select(["id", "cleaned_text"])

#Preprocessing
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') 

english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_"]

stopwords = english + time + reddit + topic_specific

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('unigrams') \
     .setStopWords(stopwords)

pos = PerceptronModel.load("/project/macs40123/spark-jars/pos_anc_en_3.0.0_3.0_1614962126490/")\
      .setInputCols("document", "unigrams")\
      .setOutputCol("pos")

finisher = Finisher().setInputCols(['unigrams', 'pos'])

my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          stopwords_cleaner,
          #ngrammer,
          pos,
          finisher
      ])

pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.persist()

#Filter by POS
def filter_unigrams(finished_unigrams, finished_pos):
    '''Filters individual words based on their POS tag'''
    return [word for word, pos in zip(finished_unigrams, finished_pos)
            if pos in ['JJ', 'NN', 'NNS', 'NNPS']]

udf_filter_unigrams = F.udf(filter_unigrams, T.ArrayType(T.StringType()))

processed_data = processed_data.withColumn('filtered_unigrams_by_pos', udf_filter_unigrams(
                                                   F.col('finished_unigrams'),
                                                   F.col('finished_pos')))

#Now that POS was done, lemmatization makes more sense at this point

#Merge tokens as just one string to be able to take it as a document in the new Pipeline
tokens_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
processed_data = processed_data.withColumn('joined_tokens', tokens_as_string(F.col('filtered_unigrams_by_pos')))

last_documentAssembler = DocumentAssembler() \
     .setInputCol('joined_tokens') \
     .setOutputCol('joined_document')

last_tokenizer = Tokenizer() \
     .setInputCols(['joined_document']) \
     .setOutputCol('tokenized')
     
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("tokenized")\
      .setOutputCol("lemmatized")

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('final') \
     .setStopWords(stopwords)

last_finisher = Finisher() \
     .setInputCols(['final']) \

last_pipeline = Pipeline() \
     .setStages([last_documentAssembler,                  
                 last_tokenizer,
                 lemmatizer,
                 stopwords_cleaner,
                 last_finisher])

final_data = last_pipeline.fit(processed_data).transform(processed_data)

processed_data.unpersist()
final_data.persist()

## Vectorization
#Apply TF-IDF filtering
tfizer = CountVectorizer(inputCol='finished_final', outputCol='tf_features', minDF=0.01, maxDF=0.80, vocabSize= 2000)
tf_model = tfizer.fit(final_data)
tf_result = tf_model.transform(final_data)

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

final_data.unpersist()
tfidf_result.persist()

## LDA
lda = LDA(k=15, maxIter=50, learningDecay=0.5, learningOffset = 50, featuresCol='tf_idf_features', seed=2503)
lda_model = lda.fit(tfidf_result)

24/11/25 21:12:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


24/11/25 21:20:02 WARN OnlineLDAOptimizer: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.
24/11/25 21:20:02 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/11/25 21:20:02 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [2]:
def evaluate_model(model, data):
    log_likelihood = model.logLikelihood(data)
    perplexity = model.logPerplexity(data)
    return log_likelihood, perplexity


In [3]:
evaluate_model(lda_model, tfidf_result)

(-174302354.053725, 5.834824923979042)

In [4]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [5]:
num_top_words = 15

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                                        |
+-----+--------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[brain, emotion, food, stress, tired, drug, effect, anger, lazy, feeling, memory, alcohol, addiction, emotional, healthy]                         |
|1    |[anxiety, story, okay, family, friend, depression, voice, child, feeling, support, boundaries, fix, opinion, hair, heart]                         |
|2    |[job, school, college, parent, class, kid, home, money, high, study, mom, degree, house, dad, university]                                         |
|3    |[fear, choice, comment, pain, mindset, perfect, project, journe

In [6]:
## LDA
lda = LDA(k=10, maxIter=50, learningDecay=0.5, learningOffset = 50, featuresCol='tf_idf_features', seed=2503)
lda_model = lda.fit(tfidf_result)

24/11/25 21:21:31 WARN OnlineLDAOptimizer: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.


In [7]:
evaluate_model(lda_model, tfidf_result)

(-176176196.4664392, 5.897552375209811)

In [8]:
num_top_words = 15

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                            |
+-----+--------------------------------------------------------------------------------------------------------------------------------------+
|0    |[brain, health, food, mental, energy, stress, emotion, tired, motivation, healthy, smoke, drug, drink, addiction, effect]             |
|1    |[anxiety, game, depression, therapy, story, family, therapist, voice, feeling, support, fix, friend, okay, video, boundaries]         |
|2    |[job, school, college, parent, money, class, kid, home, high, house, mom, degree, family, dad, study]                                 |
|3    |[fear, comment, pain, choice, op, project, mindset, perfect, shit, failure, challenge, responsibility, nobody, piece, music]          |

In [9]:
## LDA
lda3 = LDA(k=10, maxIter=50, learningDecay=0.5, learningOffset = 50, featuresCol='tf_idf_features', topicConcentration= 0.1, seed=2503)
lda_model3 = lda3.fit(tfidf_result)

24/11/25 21:22:42 WARN OnlineLDAOptimizer: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.


In [10]:
evaluate_model(lda_model3, tfidf_result)

(-176176196.46643913, 5.897552375209811)

In [11]:
num_top_words = 15

topics = lda_model3.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                            |
+-----+--------------------------------------------------------------------------------------------------------------------------------------+
|0    |[brain, health, food, mental, energy, stress, emotion, tired, motivation, healthy, smoke, drug, drink, addiction, effect]             |
|1    |[anxiety, game, depression, therapy, story, family, therapist, voice, feeling, support, fix, friend, okay, video, boundaries]         |
|2    |[job, school, college, parent, money, class, kid, home, high, house, mom, degree, family, dad, study]                                 |
|3    |[fear, comment, pain, choice, op, project, mindset, perfect, shit, failure, challenge, responsibility, nobody, piece, music]          |

In [12]:
## LDA
lda4 = LDA(k=20, maxIter=50, learningDecay=0.5, learningOffset = 50, featuresCol='tf_idf_features', topicConcentration= 0.1, seed=2503)
lda_model4 = lda4.fit(tfidf_result)
evaluate_model(lda_model4, tfidf_result)
num_top_words = 15
topics = lda_model4.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

24/11/25 21:23:53 WARN OnlineLDAOptimizer: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.


+-----+--------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                            |
+-----+--------------------------------------------------------------------------------------------------------------------------------------+
|0    |[emotion, brain, anxiety, stress, feeling, depression, emotional, drug, effect, anger, anxious, pattern, addiction, memory, tired]    |
|1    |[story, support, family, voice, weird, hair, adult, brother, future, young, boundaries, moment, accept, balance, trust]               |
|2    |[job, school, college, parent, class, kid, high, home, study, degree, money, house, university, student, country]                     |
|3    |[fear, choice, pain, mindset, comment, decision, project, growth, nobody, pressure, perfect, journey, version, challenge, constant]   |

In [13]:
## LDA
lda5 = LDA(k=14, maxIter=50, learningDecay=0.5, learningOffset = 50, featuresCol='tf_idf_features', topicConcentration= 0.05, seed=2503)
lda_model5 = lda5.fit(tfidf_result)
evaluate_model(lda_model5, tfidf_result)
num_top_words = 15
topics = lda_model5.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

24/11/25 21:25:35 WARN OnlineLDAOptimizer: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.


+-----+-------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                           |
+-----+-------------------------------------------------------------------------------------------------------------------------------------+
|0    |[brain, emotion, food, stress, tired, drug, effect, anger, healthy, lazy, feeling, alcohol, eat, memory, emotional]                  |
|1    |[anxiety, story, okay, family, friend, voice, child, feeling, fine, afraid, support, depression, boundaries, accept, heart]          |
|2    |[job, school, college, parent, class, kid, money, home, high, mom, degree, study, house, dad, family]                                |
|3    |[fear, choice, comment, mindset, pain, project, perfect, version, responsibility, growth, piece, journal, journey, failure, note]    |
|4    

In [14]:
evaluate_model(lda_model5, tfidf_result)

(-174550461.59692672, 5.843130400315031)

In [15]:
## LDA
lda5 = LDA(k=12, maxIter=50, learningDecay=0.5, learningOffset = 50, featuresCol='tf_idf_features', topicConcentration= 0.05, seed=2503)
lda_model5 = lda5.fit(tfidf_result)
print(evaluate_model(lda_model5, tfidf_result))
num_top_words = 15
topics = lda_model5.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

24/11/25 21:27:41 WARN OnlineLDAOptimizer: The input data is not directly cached, which may hurt performance if its parent RDDs are also uncached.


(-175227136.81462666, 5.86578231139856)
+-----+---------------------------------------------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                                                             |
+-----+---------------------------------------------------------------------------------------------------------------------------------------+
|0    |[brain, emotion, food, energy, stress, tired, healthy, drink, drug, effect, anger, cycle, lazy, feeling, alcohol]                      |
|1    |[anxiety, family, feeling, story, okay, friend, voice, child, angry, afraid, fine, wrong, depression, anxious, everything]             |
|2    |[job, school, college, parent, class, kid, home, money, high, house, mom, degree, study, dad, family]                                  |
|3    |[fear, comment, pain, choice, project, mindset, perfect, nobody, piece, journal, version,

Create dataframe with topic distributions

In [ ]:
# Transform data to get topic distributions
df_with_lda = lda_model.transform(tfidf_result)

df_with_lda.select("topicDistribution").show(1, truncate=False)